In [1]:
import sys
sys.path.insert(0,"/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/") 
__package__ = 'notebooks'
from utils.data_loaders import BertDataset, Wav2VecAudioDataset
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torch.utils.data import TensorDataset
import numpy as np


In [2]:
df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")
df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,label,num_channels,audio_shape,size_padding,video_path
11396,4,38,4,"oh it s great , it s a role on",10,test,../../data/test_splits_wav/dia38_utt4.wav,dia38_utt4,joy,6,4879360,29276160,../../data/test_video/dia38_utt4.mp4
12087,5,220,0,what s that smell ?,5,test,../../data/test_splits_wav/dia220_utt0.wav,dia220_utt0,disgust,6,3761152,22566912,../../data/test_video/dia220_utt0.mp4
9943,4,309,0,kathy ! kathy ! hi ! ! kathy ! kathy ! ( she d...,29,train,../../data/train_splits_wav/dia309_utt0.wav,dia309_utt0,joy,6,656725,3940350,../../data/train_video/dia309_utt0.mp4
10195,4,485,0,"hey , this heyy",4,train,../../data/train_splits_wav/dia485_utt0.wav,dia485_utt0,joy,6,412672,2476032,../../data/train_video/dia485_utt0.mp4
3411,0,757,2,"y'know , i do n't know if you 've ever looked ...",50,train,../../data/train_splits_wav/dia757_utt2.wav,dia757_utt2,neutral,6,388437,2330622,../../data/train_video/dia757_utt2.mp4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4
7031,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4
10167,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4
9618,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4


In [3]:
train_csv = pd.read_csv("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/train_sent_emo.csv")
val_csv = pd.read_csv("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/dev_sent_emo.csv")
test_csv = pd.read_csv("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/test_sent_emo.csv")

In [9]:
val_csv.columns

Index(['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Sentiment', 'Dialogue_ID',
       'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime'],
      dtype='object')

In [18]:
# val_csv
from tqdm import tqdm
tqdm.pandas()
# ../../data/train_splits_wav/dia332_utt1.wav
def aud_path(df , split):
    df['audio_path'] = df.progress_apply(lambda x: f"../../data/{split}_splits_wav/dia{x.Dialogue_ID}_utt{x.Utterance_ID}.wav" , axis = 1)
    df.drop(['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Dialogue_ID',
       'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime'], inplace = True ,axis=1)



In [19]:
aud_path(val_csv , "val")
aud_path(train_csv , "train")
aud_path(test_csv , "test")

100%|██████████| 2610/2610 [00:00<00:00, 29034.72it/s]


In [20]:
val_csv

,Sentiment,audio_path
0,negative,../../data/val_splits_wav/dia0_utt0.wav
1,negative,../../data/val_splits_wav/dia0_utt1.wav
2,neutral,../../data/val_splits_wav/dia1_utt0.wav
3,positive,../../data/val_splits_wav/dia1_utt1.wav
4,negative,../../data/val_splits_wav/dia1_utt2.wav
...,...,...
1104,negative,../../data/val_splits_wav/dia113_utt9.wav
1105,negative,../../data/val_splits_wav/dia113_utt10.wav
1106,negative,../../data/val_splits_wav/dia113_utt11.wav
1107,negative,../../data/val_splits_wav/dia113_utt12.wav


In [21]:
new_csv = pd.concat([val_csv ,
train_csv ,
test_csv ,])
new_csv

,Sentiment,audio_path
0,negative,../../data/val_splits_wav/dia0_utt0.wav
1,negative,../../data/val_splits_wav/dia0_utt1.wav
2,neutral,../../data/val_splits_wav/dia1_utt0.wav
3,positive,../../data/val_splits_wav/dia1_utt1.wav
4,negative,../../data/val_splits_wav/dia1_utt2.wav
...,...,...
2605,neutral,../../data/test_splits_wav/dia279_utt11.wav
2606,negative,../../data/test_splits_wav/dia279_utt12.wav
2607,neutral,../../data/test_splits_wav/dia279_utt13.wav
2608,neutral,../../data/test_splits_wav/dia279_utt14.wav


In [43]:
list(dict(sorted((dict((new_csv['Sentiment'].value_counts()/len(new_csv))).items()))).values())

[0.3052232273125182, 0.46950685730960023, 0.22526991537788152]

('neutral', 0.5304931426903998), ('negative', 0.6947767726874818), ('positive', 0.7747300846221185)


In [25]:
new_csv['Sentiment'].value_counts()


neutral     6436
negative    4184
positive    3088
Name: Sentiment, dtype: int64

In [33]:
new_class_df = pd.merge(df , new_csv , on = "audio_path")
new_class_df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,label,num_channels,audio_shape,size_padding,video_path,Sentiment
0,4,38,4,"oh it s great , it s a role on",10,test,../../data/test_splits_wav/dia38_utt4.wav,dia38_utt4,joy,6,4879360,29276160,../../data/test_video/dia38_utt4.mp4,positive
1,5,220,0,what s that smell ?,5,test,../../data/test_splits_wav/dia220_utt0.wav,dia220_utt0,disgust,6,3761152,22566912,../../data/test_video/dia220_utt0.mp4,negative
2,4,309,0,kathy ! kathy ! hi ! ! kathy ! kathy ! ( she d...,29,train,../../data/train_splits_wav/dia309_utt0.wav,dia309_utt0,joy,6,656725,3940350,../../data/train_video/dia309_utt0.mp4,positive
3,4,485,0,"hey , this heyy",4,train,../../data/train_splits_wav/dia485_utt0.wav,dia485_utt0,joy,6,412672,2476032,../../data/train_video/dia485_utt0.mp4,positive
4,0,757,2,"y'know , i do n't know if you 've ever looked ...",50,train,../../data/train_splits_wav/dia757_utt2.wav,dia757_utt2,neutral,6,388437,2330622,../../data/train_video/dia757_utt2.mp4,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13699,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4,negative
13700,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4,negative
13701,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4,positive
13702,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4,positive


In [34]:
codes = {'negative':0, 'neutral':1, 'positive':2}


In [35]:
new_class_df['sentiment'] = new_class_df["Sentiment"].map(codes)
new_class_df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,label,num_channels,audio_shape,size_padding,video_path,Sentiment,sentiment
0,4,38,4,"oh it s great , it s a role on",10,test,../../data/test_splits_wav/dia38_utt4.wav,dia38_utt4,joy,6,4879360,29276160,../../data/test_video/dia38_utt4.mp4,positive,2
1,5,220,0,what s that smell ?,5,test,../../data/test_splits_wav/dia220_utt0.wav,dia220_utt0,disgust,6,3761152,22566912,../../data/test_video/dia220_utt0.mp4,negative,0
2,4,309,0,kathy ! kathy ! hi ! ! kathy ! kathy ! ( she d...,29,train,../../data/train_splits_wav/dia309_utt0.wav,dia309_utt0,joy,6,656725,3940350,../../data/train_video/dia309_utt0.mp4,positive,2
3,4,485,0,"hey , this heyy",4,train,../../data/train_splits_wav/dia485_utt0.wav,dia485_utt0,joy,6,412672,2476032,../../data/train_video/dia485_utt0.mp4,positive,2
4,0,757,2,"y'know , i do n't know if you 've ever looked ...",50,train,../../data/train_splits_wav/dia757_utt2.wav,dia757_utt2,neutral,6,388437,2330622,../../data/train_video/dia757_utt2.mp4,neutral,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13699,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4,negative,0
13700,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4,negative,0
13701,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4,positive,2
13702,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4,positive,2


In [36]:
new_class_df.rename(columns={"label": "emotion_label", "Sentiment": "sentiment_label"} , inplace=True)

In [37]:
new_class_df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment
0,4,38,4,"oh it s great , it s a role on",10,test,../../data/test_splits_wav/dia38_utt4.wav,dia38_utt4,joy,6,4879360,29276160,../../data/test_video/dia38_utt4.mp4,positive,2
1,5,220,0,what s that smell ?,5,test,../../data/test_splits_wav/dia220_utt0.wav,dia220_utt0,disgust,6,3761152,22566912,../../data/test_video/dia220_utt0.mp4,negative,0
2,4,309,0,kathy ! kathy ! hi ! ! kathy ! kathy ! ( she d...,29,train,../../data/train_splits_wav/dia309_utt0.wav,dia309_utt0,joy,6,656725,3940350,../../data/train_video/dia309_utt0.mp4,positive,2
3,4,485,0,"hey , this heyy",4,train,../../data/train_splits_wav/dia485_utt0.wav,dia485_utt0,joy,6,412672,2476032,../../data/train_video/dia485_utt0.mp4,positive,2
4,0,757,2,"y'know , i do n't know if you 've ever looked ...",50,train,../../data/train_splits_wav/dia757_utt2.wav,dia757_utt2,neutral,6,388437,2330622,../../data/train_video/dia757_utt2.mp4,neutral,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13699,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4,negative,0
13700,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4,negative,0
13701,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4,positive,2
13702,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4,positive,2


In [38]:
new_class_df.to_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")

In [2]:
df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")
df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
1,0,243,3,"okay , very cute braces anyway y know what , the",11,test,../../data/test_splits_wav/dia243_utt3.wav,dia243_utt3,neutral,6,166229,997374,../../data/test_video/dia243_utt3.mp4,neutral,1,"(1.100986873795761, 5.204665221579961)"
12,0,64,8,"ah , gunther , i can t pay for this right now ...",34,test,../../data/test_splits_wav/dia64_utt8.wav,dia64_utt8,neutral,6,164181,985086,../../data/test_video/dia64_utt8.mp4,neutral,1,"(0.280582763671875, 8.89848193359375)"
28,0,241,3,"listen uh , not that i m y know insecure about...",32,test,../../data/test_splits_wav/dia241_utt3.wav,dia241_utt3,neutral,6,161792,970752,../../data/test_video/dia241_utt3.mp4,neutral,1,"(0.7208554455445544, 9.69150099009901)"
43,0,22,2,i just came over to drop off nothing so that w...,16,test,../../data/test_splits_wav/dia22_utt2.wav,dia22_utt2,neutral,6,160427,962562,../../data/test_video/dia22_utt2.mp4,neutral,1,"(0.22014683133732535, 9.866580713572855)"
48,0,7,7,"huh , what s fish hooking thanks man , that wo...",22,test,../../data/test_splits_wav/dia7_utt7.wav,dia7_utt7,neutral,6,159061,954366,../../data/test_video/dia7_utt7.mp4,neutral,1,"(0.46098828125, 8.337875)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,6,37,7,oh ! oh !,4,val,../../data/val_splits_wav/dia37_utt7.wav,dia37_utt7,disgust,6,20139,120834,../../data/val_video/dia37_utt7.mp4,negative,0,"(0.6496451612903226, 1.1774818548387098)"
11352,6,91,8,no !,2,val,../../data/val_splits_wav/dia91_utt8.wav,dia91_utt8,disgust,6,16384,98304,../../data/val_video/dia91_utt8.mp4,negative,0,"(0.08192, 0.22528)"
12740,6,7,8,she ll drive us totally crazy,6,val,../../data/val_splits_wav/dia7_utt8.wav,dia7_utt8,disgust,2,25259,50518,../../data/val_video/dia7_utt8.mp4,negative,0,"(0.16191666666666665, 1.4774895833333332)"
12772,6,20,5,this is disgusting !,4,val,../../data/val_splits_wav/dia20_utt5.wav,dia20_utt5,disgust,6,8192,49152,../../data/val_video/dia20_utt5.mp4,negative,0,"(0.04096, 0.49151999999999996)"


In [3]:
df = df[ (df['emotion_label'] != "fear")  & (df['emotion_label'] != "disgust")]
# df = df[~df['timings'].isna()]
df = df[ (df['split'] == "train") ]

In [4]:
def col(batch , df):
    """
    Here we are going to take some raw-input and pre-process them into what we need
    So we can see all the steps in the ML as they go along
    """
    speech_list = []
    label_list = []

    for (input , label) in batch:
        speech_list.append(input)
        label_list.append(label)
        # print( f"For path {input}: ", label , " =/= " , df[df['audio_path'] == input]['emotion'].values[0] , flush= True)
        assert label == df[df['audio_path'] == input]['emotion'].values[0]
    
   
    
    return speech_list , np.array(label_list)

In [8]:
labels = df['emotion'].value_counts()

# Create sampler, dataset, loader

# train_dataset = BertDataset(df.sample(frac = 1), max_len=70 , feature_col="text" , label_col = "emotion")
train_dataset = Wav2VecAudioDataset(df , feature_col="audio_path" , label_col = 'emotion')

class_counts = torch.Tensor(list(dict(sorted((dict((labels)).items()))).values())).to(int)

bs = 170

samples_weight = torch.tensor([1/class_counts[t] for t in df['emotion']])
print(samples_weight, "\n\n" , len(samples_weight))
sampler = WeightedRandomSampler(list(samples_weight), len(samples_weight))
#train_dataset = triaxial_dataset(data, target)
train_loader = DataLoader(
    train_dataset, batch_size=bs, num_workers=0, sampler=sampler , drop_last=True, collate_fn= lambda x: col(x , df=df))

for i, (x, y) in enumerate(train_loader):
    try:
        print(f"batch index {i}, 0/1/2/3/4: {(y == 0).sum()}/{(y == 1).sum()}/{(y == 2).sum()}/{(y == 3).sum()}/{(y == 4).sum()}")
    except:
        print(y)

# Iterate DataLoader and check class balance for each batch


tensor([0.0002, 0.0002, 0.0002,  ..., 0.0015, 0.0015, 0.0015]) 

 9307
batch index 0, 0/1/2/3/4: 44/35/31/36/24
batch index 1, 0/1/2/3/4: 39/40/23/34/34
batch index 2, 0/1/2/3/4: 33/35/29/35/38
batch index 3, 0/1/2/3/4: 38/40/28/32/32
batch index 4, 0/1/2/3/4: 33/32/32/40/33
batch index 5, 0/1/2/3/4: 29/38/35/33/35
batch index 6, 0/1/2/3/4: 35/36/30/29/40
batch index 7, 0/1/2/3/4: 32/33/43/28/34
batch index 8, 0/1/2/3/4: 31/31/39/34/35
batch index 9, 0/1/2/3/4: 42/32/34/29/33
batch index 10, 0/1/2/3/4: 31/27/40/33/39
batch index 11, 0/1/2/3/4: 41/32/33/35/29
batch index 12, 0/1/2/3/4: 35/37/27/36/35
batch index 13, 0/1/2/3/4: 29/28/40/37/36
batch index 14, 0/1/2/3/4: 29/36/27/45/33
batch index 15, 0/1/2/3/4: 36/30/32/37/35
batch index 16, 0/1/2/3/4: 36/26/37/41/30
batch index 17, 0/1/2/3/4: 28/31/36/41/34
batch index 18, 0/1/2/3/4: 39/41/33/26/31
batch index 19, 0/1/2/3/4: 34/34/30/32/40
batch index 20, 0/1/2/3/4: 36/35/32/38/29
batch index 21, 0/1/2/3/4: 21/40/42/29/38
batch index 22,

In [27]:
df[df['audio_path']=="../../data/train_splits_wav/dia615_utt17.wav"]

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
4175,0,615,17,"oh , now , do n't listen to him , pheebs , i t...",17,train,../../data/train_splits_wav/dia615_utt17.wav,dia615_utt17,neutral,6,50176,301056,../../data/train_video/dia615_utt17.mp4,neutral,1,"(0.060307692307692305, 3.035487179487179)"
